# Pipelines and models

In [1]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)

# imports
from common.utils import *

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = get_data()

X = data.drop("Target", axis=1)
y = data["Target"]

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,single,2nd phase - general contingent,6th choice,Animation and Multimedia Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 3rd Cycle,Other - 11th Year,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,single,International student (bachelor),2nd choice,Tourism,Daytime,Secondary education,160.0,Portuguese,Secondary Education - 12th Year or Eq.,Higher Ed - Degree,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,single,1st phase - general contingent,6th choice,Communication Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 1st Cycle (4th/5th),...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,single,2nd phase - general contingent,3rd choice,Journalism and Communication,Daytime,Secondary education,122.0,Portuguese,Basic Ed 2nd Cycle (6th–8th),Basic Ed 1st Cycle (4th/5th),...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,married,Over 23 years old,2nd choice,Social Service (evening attendance),Evening,Secondary education,100.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 2nd Cycle (6th–8th),...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [3]:
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),                  # strategy="mean"
    ("scaler", StandardScaler()), 
])

categoric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),           # strategy="constant"
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),
        ("cat", categoric_pipeline, categorical_features)
    ]
)

In [4]:
model = LogisticRegression(max_iter=250)
get_custom_dataframe(model, preprocessor, accuracy_score, X_train_val, y_train_val, X_test, y_test)

,0,1,2,3,4
Train,0.809961,0.809255,0.809961,0.819145,0.809255
Validation,0.774011,0.779661,0.793785,0.742938,0.776836
Test,0.779661,0.786441,0.776271,0.785311,0.772881


In [5]:
model = DecisionTreeClassifier(max_depth = 7, min_samples_split=10, min_samples_leaf=3)
get_custom_dataframe(model, preprocessor, accuracy_score, X_train_val, y_train_val, X_test, y_test)

,0,1,2,3,4
Train,0.800071,0.811727,0.802543,0.801130,0.806076
Validation,0.737288,0.721751,0.764124,0.730226,0.724576
Test,0.745763,0.743503,0.762712,0.727684,0.740113


In [6]:
model = SVC()
get_custom_dataframe(model, preprocessor, accuracy_score, X_train_val, y_train_val, X_test, y_test)

,0,1,2,3,4
Train,0.836454,0.839986,0.841046,0.836454,0.830095
Validation,0.782486,0.745763,0.745763,0.757062,0.778249
Test,0.780791,0.784181,0.781921,0.778531,0.784181
